In [1]:
import ibis
import gcsfs

from ibis_bench.utils.monitor import get_timings_dir

ibis.options.interactive = True
ibis.options.repr.interactive.max_rows = 20
ibis.options.repr.interactive.max_columns = None

In [2]:
# YOLO
import warnings

warnings.filterwarnings("ignore")

In [3]:
PROJECT = "voltrondata-demo"
BUCKET = "ibis-benchy"

fs = gcsfs.GCSFileSystem(project=PROJECT)
fs.ls(f"{BUCKET}/{get_timings_dir()}")[-5:]

['ibis-benchy/benchy_logs_v2/d53be7c2-3b19-46d9-95e5-0d53d7b48171.json',
 'ibis-benchy/benchy_logs_v2/dd136bd6-8384-4f4f-884e-50317b18a61d.json',
 'ibis-benchy/benchy_logs_v2/e55be847-7ea4-4b40-9ca7-380363a5ac13.json',
 'ibis-benchy/benchy_logs_v2/e75e7e09-046a-436d-904d-d5ca4e69fd02.json',
 'ibis-benchy/benchy_logs_v2/f74cda0c-33cd-4d38-b009-08d431faab86.json']

In [4]:
con = ibis.connect("duckdb://")
con.register_filesystem(fs)

In [5]:
t = (
    con.read_json(f"gs://{BUCKET}/{get_timings_dir()}/*.json")
    .mutate(
        timestamp=ibis._["timestamp"].cast("timestamp"),
    )
    .cache()
)
t

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ session_id                           ┃ system          ┃ timestamp                  ┃ sf    ┃ n_partitions ┃ query_number ┃ execution_seconds ┃ peak_cpu      ┃ peak_memory ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ uuid                                 │ string          │ timestamp(6)               │ int64 │ int64        │ int64        │ float64           │ float64       │ float64     │
├──────────────────────────────────────┼─────────────────┼────────────────────────────┼───────┼──────────────┼──────────────┼───────────────────┼───────────────┼─────────────┤
│ 245480c9-2d7b-473a-a81c-a868c7325503 │ ibis-polars     │ 2024-06-01 01:21:12.864228 │   100 │            1 │            1 │        162.723401 │     96.114019 │    0.000284 │
│ 245480c9-2d7b-473a-a81c-a868c7325503 │ ibis-polars     │ 2024-06-01 01:18:28.371871 │    10 │            1 │            2 │          0.992329 │  13745.436307 │    0.000692 │
│ 40a7b0d9-cda4-4734-bb90-0a5b2536abcc │ ibis-duckdb     │ 2024-06-01 01:17:16.521537 │    10 │            1 │            2 │          1.106904 │  12033.560970 │    0.001241 │
│ d3651099-13a8-440a-8216-e2245f73f0b6 │ polars-lazy     │ 2024-06-01 01:22:01.891880 │    10 │            1 │            1 │          2.468100 │  21230.905755 │    0.000006 │
│ 7709c520-ce9e-4563-adc8-9872c0c7837d │ ibis-duckdb     │ 2024-06-01 01:43:08.303382 │     8 │            1 │            1 │          0.743626 │ 106747.226275 │    0.000346 │
│ 7709c520-ce9e-4563-adc8-9872c0c7837d │ ibis-duckdb     │ 2024-06-01 01:43:17.884621 │    32 │            1 │            1 │          2.738984 │  30909.270249 │    0.000169 │
│ d3651099-13a8-440a-8216-e2245f73f0b6 │ polars-lazy     │ 2024-06-01 01:21:59.047134 │     1 │            1 │            2 │          0.048498 │ 420842.831846 │    0.000004 │
│ 7709c520-ce9e-4563-adc8-9872c0c7837d │ ibis-duckdb     │ 2024-06-01 01:43:14.420332 │    16 │            1 │            2 │          1.160468 │  14907.780345 │    0.001241 │
│ 3856c658-a39b-412c-9271-1c89c4743f31 │ ibis-datafusion │ 2024-06-01 01:17:48.541654 │    10 │            1 │            2 │          1.018333 │  15054.015365 │    0.001170 │
│ 40a7b0d9-cda4-4734-bb90-0a5b2536abcc │ ibis-duckdb     │ 2024-06-01 01:17:10.704071 │     1 │            1 │            1 │          0.157612 │ 332589.212189 │    0.000346 │
│ 40a7b0d9-cda4-4734-bb90-0a5b2536abcc │ ibis-duckdb     │ 2024-06-01 01:17:15.099789 │    10 │            1 │            1 │          0.898457 │  90889.152654 │    0.000169 │
│ 245480c9-2d7b-473a-a81c-a868c7325503 │ ibis-polars     │ 2024-06-01 01:18:27.081115 │    10 │            1 │            1 │          2.005881 │  31218.201768 │    0.000081 │
│ d3651099-13a8-440a-8216-e2245f73f0b6 │ polars-lazy     │ 2024-06-01 01:24:26.631866 │   100 │            1 │            1 │        144.008452 │    129.228526 │    0.000249 │
│ 245480c9-2d7b-473a-a81c-a868c7325503 │ ibis-polars     │ 2024-06-01 01:18:21.605979 │     1 │            1 │            1 │          0.280429 │ 160254.532173 │    0.002115 │
│ d3651099-13a8-440a-8216-e2245f73f0b6 │ polars-lazy     │ 2024-06-01 01:24:29.640294 │   100 │            1 │            2 │          2.429198 │  10402.610130 │    0.000004 │
│ 3856c658-a39b-412c-9271-1c89c4743f31 │ ibis-datafusion │ 2024-06-01 01:17:58.464738 │   100 │            1 │            1 │          9.525200 │   9418.174819 │    0.000277 │
│ 245480c9-2d7b-473a-a81c-a868c7325503 │ ibis-polars     │ 2024-06-01 01:18:24.685265 │     1 │            1 │            2 │          0.933135 │  11348.842702 │    0.000733 │
│ 7709c520-ce9e-4563-adc8-9872c0c7837d │ ibis-duckdb     │ 2024-06-01 01:43:10.822048 │     8 │            1 │            2 │   

In [6]:
t.count()

30

In [7]:
(
    t.filter(t["sf"] >= 1)
    .filter((t["system"].contains("duckdb")) | (t["system"].contains("datafusion")))
    .filter(t["query_number"] == 1)
    .group_by("system", "sf", "n_partitions", "query_number")
    .agg(
        mean_execution_seconds=t["execution_seconds"].mean(),
        max_peak_cpu=t["peak_cpu"].max(),
        max_peak_memory=t["peak_memory"].max(),
    )
    .order_by(
        ibis.desc("sf"),
        ibis.asc("n_partitions"),
        ibis.desc("system"),
        ibis.asc("query_number"),
        ibis.asc("mean_execution_seconds"),
    )
)

┏━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ system          ┃ sf    ┃ n_partitions ┃ query_number ┃ mean_execution_seconds ┃ max_peak_cpu  ┃ max_peak_memory ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string          │ int64 │ int64        │ int64        │ float64                │ float64       │ float64         │
├─────────────────┼───────┼──────────────┼──────────────┼────────────────────────┼───────────────┼─────────────────┤
│ ibis-duckdb     │   100 │            1 │            1 │               8.436633 │  10232.755043 │        0.000169 │
│ ibis-datafusion │   100 │            1 │            1 │               9.525200 │   9418.174819 │        0.000277 │
│ ibis-duckdb     │    32 │            1 │            1 │               2.738984 │  30909.270249 │        0.000169 │
│ ibis-duckdb     │    16 │            1 │            1 │               1.407763 │  59086.650058 │        0.000169 │
│ ibis-duckdb     │    10 │            1 │            1 │               0.898457 │  90889.152654 │        0.000169 │
│ ibis-datafusion │    10 │            1 │            1 │               1.037822 │  80244.973943 │        0.000297 │
│ ibis-duckdb     │     8 │            1 │            1 │               0.743626 │ 106747.226275 │        0.000346 │
│ ibis-duckdb     │     1 │            1 │            1 │               0.157612 │ 332589.212189 │        0.000346 │
│ ibis-datafusion │     1 │            1 │            1 │               0.230627 │ 188832.806483 │        0.002122 │
└─────────────────┴───────┴──────────────┴──────────────┴────────────────────────┴───────────────┴─────────────────┘

In [8]:
t["session_id"].value_counts()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ session_id                           ┃ session_id_count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ uuid                                 │ int64            │
├──────────────────────────────────────┼──────────────────┤
│ 245480c9-2d7b-473a-a81c-a868c7325503 │                6 │
│ 3856c658-a39b-412c-9271-1c89c4743f31 │                6 │
│ 7709c520-ce9e-4563-adc8-9872c0c7837d │                6 │
│ 40a7b0d9-cda4-4734-bb90-0a5b2536abcc │                6 │
│ d3651099-13a8-440a-8216-e2245f73f0b6 │                6 │
└──────────────────────────────────────┴──────────────────┘